# Глубинное обучение для текстовых данных, ФКН ВШЭ
## Домашнее задание 4: Уменьшение размеров модели
### Оценивание и штрафы

Максимально допустимая оценка за работу — __10 баллов__.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн 29.11.24 23:59__ \
__Жесткий дедлайн 2.12.24 23:59__

### О задании

В этом задании вам предстоит научиться решать задачу Named Entity Recognition (NER) на самом популярном датасете – [CoNLL-2003](https://paperswithcode.com/dataset/conll-2003). В вашем распоряжении будет предобученный BERT, который вам необходимо уменьшить с минимальными потерями в качестве до размера 20М параметров. Для этого вы самостоятельно реализуете факторизацию эмбеддингов, дистилляцию, шеринг параметров и так далее.

В этом задании вам придется проводить довольно много экспериментов, поэтому мы рекомендуем не писать весь код в тетрадке, а завести разные файлы для отдельных логических блоков и скомпоновать все в виде проекта. Это позволит вашему ноутбуку не разрастаться и сильно облегчит задачу и вам, и проверяющим. Так же постарайтесь логгировать все ваши эксперименты в wandb, чтобы ничего не потерялось.

### Оценивание
Оценка за это домашнее задание будет формироваться из оценки за __задания__ и за __отчет__, в котором, как и раньше, от вас требуется написать о проделанной работе. За отчет можно получить до 2-х баллов, однако в случае отсутствия отчета можно потерять баллы за сами задания. Задания делятся на две части: _номерные_ и _на выбор_. За _номерные_ можно получить в сумме 6 баллов, за задания _на выбор_ можно получить до 16. То есть за все дз можно получить 24 балла. Все, что вы наберете свыше 10, будет считаться бонусами.


### О датасете

Named Entity Recognition – это задача классификации токенов по классам сущностей. В CoNLL-2003 для именования сущностей используется маркировка **BIO** (Beggining, Inside, Outside), в которой метки означают следующее:

- *B-{метка}* – начало сущности *{метка}*
- *I-{метка}* – продолжнение сущности *{метка}*
- *O* – не сущность

Существуют так же и другие способы маркировки, например, BILUO. Почитать о них можно [тут](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging)) и [тут](https://www.youtube.com/watch?v=dQw4w9WgXcQ).

Всего в датасете есть 9 разных меток.
- O – слову не соответствует ни одна сущность.
- B-PER/I-PER – слово или набор слов соответстует определенному _человеку_.
- B-ORG/I-ORG – слово или набор слов соответстует определенной _организации_.
- B-LOC/I-LOC – слово или набор слов соответстует определенной _локации_.
- B-MISC/I-MISC – слово или набор слов соответстует сущности, которая не относится ни к одной из предыдущих. Например, национальность, произведение искусства, мероприятие и т.д.

Приступим!

Начнем с загрузки и предобработки датасета.

In [1]:
%pip install gdown
%pip install seqeval


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import warnings
import wandb
import torch
import math
import numpy as np

from torch import nn
from tqdm import tqdm
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import get_scheduler, AdamW, get_scheduler, default_data_collator

warnings.filterwarnings("ignore")

wandb.login(key='46c3b8e339b3fb22dc286204510c8af5b2c3e2e5')

dataset = load_dataset("eriktks/conll2003",trust_remote_code=True)

dataset = dataset.remove_columns(["id", "pos_tags", "chunk_tags"])

/home/boris/micromamba/envs/nlp_hw4/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bspanfilov. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/boris/.netrc


In [3]:
dataset['train'][0]

{'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [4]:
label_names = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [5]:
words = dataset["train"][0]["tokens"]
label_ids = dataset["train"][0]["ner_tags"]

for i in range(len(words)):
    print(f'{words[i]}\t{label_names[label_ids[i]]}')

EU	B-ORG
rejects	O
German	B-MISC
call	O
to	O
boycott	O
British	B-MISC
lamb	O
.	O


### Предобработка

На протяжении всего домашнего задания мы будем использовать _cased_ версию BERT, то есть токенизатор будет учитывать регистр слов. Для задачи NER регистр важен, так как имена и названия организаций или предметов искусства часто пишутся с большой буквы, и будет глупо прятать от модели такую информацию.

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

При токенизации слова могут разделиться на несколько токенов (как слово `Fischler` из примера ниже), из-за чего появится несоответствие между числом токенов и меток. Это несоответствие нам придется устранить вручную.

In [7]:
example = dataset["train"][12]
words = example["tokens"]
tags = [label_names[t] for t in example["ner_tags"]]
tokenized_text = tokenizer(example["tokens"], is_split_into_words=True)


print('Слова: ', words)
print('Токены:', tokenized_text.tokens())
print('Метки:', tags)

Слова:  ['Only', 'France', 'and', 'Britain', 'backed', 'Fischler', "'s", 'proposal', '.']
Токены: ['[CLS]', 'Only', 'France', 'and', 'Britain', 'backed', 'Fi', '##sch', '##ler', "'", 's', 'proposal', '.', '[SEP]']
Метки: ['O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-PER', 'O', 'O', 'O']


__Задание 1 (1 балл).__ Токенизируйте весь датасет и для каждого текста выравните токены с метками так, чтобы каждому токену соответствовала одна метка. При этом важно сохранить нотацию BIO. И не забудьте про специальные токены! Должно получиться что-то такое:

In [8]:
def align_labels_with_tokens(sample):
    ner_tags = sample['ner_tags']
    tokenized_text = tokenizer(sample["tokens"], truncation=True, is_split_into_words=True, padding='max_length', max_length=128) # чтобы по батчам учиться можно было
    tokens = tokenized_text.tokens()
    word_ids = tokenized_text.word_ids()
    aligned_labels = []
    prev_id = -1
    for id, token in zip(word_ids, tokens):
        if id is None:
            aligned_labels.append(-100)
        elif id != prev_id or ner_tags[id] == 0 or ner_tags[id] % 2 == 0:
            aligned_labels.append(ner_tags[id])
        else:
            aligned_labels.append(ner_tags[id] + 1) # B -> I         

        prev_id = id
    tokenized_text['labels'] = aligned_labels
    return tokenized_text

In [9]:
tok_example = align_labels_with_tokens(example)
tags = [label_names[t] if t > -1 else t for t in tok_example['labels']]
print("Выровненные метки:", tok_example['labels'])
print("Выровненные названия меток:", tags)

Выровненные метки: [-100, 0, 5, 0, 5, 0, 1, 2, 2, 0, 0, 0, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
Выровненные названия меток: [-100, 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -

In [10]:
# проделаем это со всем датасетом и сохраним
tokenized_datasets = dataset.map(align_labels_with_tokens, remove_columns=["tokens", "ner_tags"])

### Метрика

Для оценки качества NER обычно используют F1 меру с микро-усреднением. Мы загрузим ее из библиотеки `seqeval`. Функция `f1_score` принимает два 2d списка с правильными и предсказанными метками, записаными текстом, и возвращает для них значение F1. Вы можете использовать ее с параметрами по умолчанию.

In [11]:
from seqeval.metrics import f1_score

Особенность подсчета F1 для NER заключается в том, что в некоторых ситуациях неправильные ответы могут засчитываться как правильные. Например, если модель предсказала `['I-PER', 'I-PER']`, то мы можем догадаться, что на самом деле должно быть `['B-PER', 'I-PER']`, так как сущность не может начинаться с `I-`. Функция `f1_score` учитывает это и поэтому работает только с текстовыми представлениями меток.

### Модель

В качестве базовой модели мы возьмем `bert-base-cased`. Как вы понимаете, он не обучался на задачу NER. Поэтому прежде чем приступать к уменьшению размера BERT, его необходимо дообучить.

__Задание 2 (1 балл)__ Дообучите `bert-base-cased` на нашем датасете с помощью обычного fine-tuning. У вас должно получиться хотя бы 0.9 F1 на тестовой выборке. Заметьте, что чем выше качество большой модели, тем лучше будет работать дистиллированный ученик. Для обучения можно использовать `Trainer` из Hugging Face.

In [12]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(label_names))

print('Число параметров:', sum(p.numel() for p in model.parameters()))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Число параметров: 107726601


In [13]:
def compute_metrics(p):
    predictions, label_ids = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, label_ids)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, label_ids)
    ]

    return {'f1': f1_score(y_pred=true_predictions, y_true=true_labels)}

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=5,
    logging_dir="./logs",
    report_to="wandb",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_steps=50,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
wandb.init(
    project="nlp_hw4_size_reduction",
    name='simple_ft'
)

trainer.train()
wandb.finish()

In [ ]:
test_results = trainer.evaluate(tokenized_datasets["test"])
print(test_results)

### Факторизация матрицы эмбеддингов

Можно заметить, что на данный момент матрица эмбеддингов занимает $V \cdot H = 28996 \cdot 768 = 22.268.928$ параметров. Это aж пятая часть от всей модели! Давайте попробуем что-то с этим сделать. В модели [ALBERT](https://arxiv.org/pdf/1909.11942.pdf) предлагается факторизовать матрицу эмбеддингов в произведение двух небольших матриц. Таким образом, параметры эмбеддингов будут содержать $V \cdot E + E \cdot H$ элементов, что гораздо меньше $V \cdot H$, если $H \gg E$. Авторы выбирают $E = 128$, однако ничего не мешает нам взять любое другое значение. Например, выбрав $H = 64$, мы уменьшим число параметров примерно на 20М.

__Задание 3 (1 балл).__ Напишите класс-обертку над слоем эмбеддингов, который реализует факторизацию на две матрицы, и дообучите факторизованную модель. Заметьте, обе матрицы можно инициализировать с помощью SVD разложения, чтобы начальное приближение было хорошим. Это сэкономит очень много времени на дообучении. С рангом разложения $H = 64$ у вас должно получиться F1 больше 0.87.

In [17]:
class FactorizedEmbedding(nn.Module):
    def __init__(self, original_embeddings, factor_dim):
        super().__init__()
        self.vocab_size, self.hidden_dim = original_embeddings.weight.size()
        self.factor_dim = factor_dim

        original_weight = original_embeddings.weight.detach().cpu().numpy()

        U, S, Vt = np.linalg.svd(original_weight, full_matrices=False)
        U = U[:, :factor_dim]
        S = np.diag(S[:factor_dim])
        V = np.dot(S, Vt[:factor_dim, :])

        self.embedding1 = nn.Embedding(self.vocab_size, factor_dim)
        self.embedding2 = nn.Linear(factor_dim, self.hidden_dim, bias=False)

        self.embedding1.weight.data = torch.tensor(U, dtype=torch.float32)
        self.embedding2.weight.data = torch.tensor(V.T, dtype=torch.float32)

    def forward(self, input_ids):
        x = self.embedding1(input_ids)  # [batch_size, seq_len, factor_dim]
        x = self.embedding2(x)  # [batch_size, seq_len, hidden_dim]
        return x

In [ ]:
factor_dim = 64
original_embeddings = model.bert.embeddings.word_embeddings
factorized_embeddings = FactorizedEmbedding(original_embeddings, factor_dim)

model.bert.embeddings.word_embeddings = factorized_embeddings

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
wandb.init(
    project="nlp_hw4_size_reduction",
    name='factorized_ft'
)

trainer.train()

In [ ]:
test_results = trainer.evaluate(tokenized_datasets["test"])
print(test_results)

wandb.finish()

### Дистилляция знаний

Дистилляция знаний – это парадигма обучения, в которой знания модели-учителя дистиллируются в модель-ученика. Учеником может быть произвольная модель меньшего размера, решающая ту же задачу, однако обычно ученик имеет ту же архитектуру, что и учитель. При дистилляции используются два функционала ошибки:

1. Стандартная кросс-энтропия.
1. Функция, задающая расстояние между распределениями предсказаний учителя и ученика. Чаще всего используют KL-дивергенцию.

Для того, чтобы распределение предсказаний учителя не было вырожденным, к softmax добавляют температуру больше 1, например, 2 или 5.   
__Важно:__ при делении логитов на температуру значения градиентов уменьшаются в $\tau^2$ раз (проверьте это!). Поэтому для возвращения их в изначальный масштаб ошибку надо домножить на $\tau^2$. Подробнее об этом можно почитать в разделе 2.1 [оригинальной статьи](https://arxiv.org/pdf/1503.02531).

<img src="https://intellabs.github.io/distiller/imgs/knowledge_distillation.png" width="1000">

__Задание 4 (3 балла).__ Реализуйте метод дистилляции знаний, изображенный на картинке. Для подсчета ошибки между предсказаниями ученика и учителя используйте KL-дивергенцию [`nn.KLDivLoss(reduction="batchmean")`](https://pytorch.org/docs/stable/generated/torch.nn.KLDivLoss.html) (обратите внимание на формат ее входов). Для получения итоговой ошибки суммируйте мягкую ошибку с жесткой.   
В качестве учителя используйте дообученный BERT из задания 2. В качестве ученика возьмите необученную модель с размером __не больше 20M__ параметров. Вы можете использовать факторизацию матрицы эмбеддингов для уменьшения числа параметров. Если вы все сделали правильно, то на тестовой выборке вы должны получить значение F1 не меньше 0.7. Вам должно хватить примерно 20к итераций обучения для этого. Если у вас что-то не получается, то можно ориентироваться на статью про [DistilBERT](https://arxiv.org/abs/1910.01108) и на [эту статью](https://www.researchgate.net/publication/375758425_Knowledge_Distillation_Scheme_for_Named_Entity_Recognition_Model_Based_on_BERT).

__Важно:__
* Не забывайте добавлять _warmup_ при обучении ученика.
* Не забывайте переводить учителя в режим _eval_.

In [ ]:
import gdown

gdown.download(id='13YDJ3EA3iBisqPcaXf7g4fdgNOixiAoY') # тут фт модель из 2го задания

In [ ]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, mask=None):
        d_k = query.size(-1)
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            mask = mask[:, None, :, None] * mask[:, None, None, :]
            ones = torch.ones(mask.shape[-2:], device=mask.device)
            mask = mask + ones
            scores = scores.masked_fill(mask == 0, -1e9)
        attention = torch.softmax(scores, dim=-1)
        attention = self.dropout(attention)
        return torch.matmul(attention, value), attention


class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_dim, num_heads, dropout=0.1):
        super().__init__()
        assert hidden_dim % num_heads == 0, "hidden_dim must be divisible by num_heads"
        self.d_k = hidden_dim // num_heads
        self.num_heads = num_heads

        self.query = nn.Linear(hidden_dim, hidden_dim)
        self.key = nn.Linear(hidden_dim, hidden_dim)
        self.value = nn.Linear(hidden_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim, hidden_dim)

        self.attention = ScaledDotProductAttention(dropout)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(hidden_dim)

    def forward(self, x, mask=None):
        batch_size, seq_len, hidden_dim = x.size()

        query = self.query(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        key = self.key(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        value = self.value(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)

        attn_output, _ = self.attention(query, key, value, mask)
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_len, hidden_dim)
        output = self.out(attn_output)

        return self.layer_norm(x + self.dropout(output))


class FeedForward(nn.Module):
    def __init__(self, hidden_dim, ff_dim, dropout=0.1):
        super().__init__()
        self.linear1 = nn.Linear(hidden_dim, ff_dim)
        self.linear2 = nn.Linear(ff_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(hidden_dim)

    def forward(self, x):
        ff_output = self.linear2(self.dropout(torch.relu(self.linear1(x))))
        return self.layer_norm(x + self.dropout(ff_output))


class TransformerEncoderLayer(nn.Module):
    def __init__(self, hidden_dim, num_heads, ff_dim, dropout=0.1):
        super().__init__()
        self.attention = MultiHeadAttention(hidden_dim, num_heads, dropout)
        self.feed_forward = FeedForward(hidden_dim, ff_dim, dropout)

    def forward(self, x, mask=None):
        x = self.attention(x, mask)
        return self.feed_forward(x)


class BERTLikeModel(nn.Module):
    def __init__(self, vocab_size, max_position_embeddings, hidden_dim, num_heads, ff_dim, num_layers, num_labels, dropout=0.1):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, hidden_dim)
        self.position_embeddings = nn.Embedding(max_position_embeddings, hidden_dim)
        self.layer_norm = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(dropout)

        self.encoder_layers = nn.ModuleList(
            [TransformerEncoderLayer(hidden_dim, num_heads, ff_dim, dropout) for _ in range(num_layers)]
        )

        self.classifier = nn.Linear(hidden_dim, num_labels)

    def forward(self, input_ids, attention_mask=None):
        batch_size, seq_len = input_ids.size()
        position_ids = torch.arange(seq_len, device=input_ids.device).unsqueeze(0).expand(batch_size, seq_len)

        x = self.embeddings(input_ids) + self.position_embeddings(position_ids)
        x = self.layer_norm(x)
        x = self.dropout(x)

        for layer in self.encoder_layers:
            x = layer(x, attention_mask)

        logits = self.classifier(x)
        return logits
    
    def __str__(self):
        """
        Model prints with the number of parameters.
        """
        all_parameters = sum([p.numel() for p in self.parameters()])
        trainable_parameters = sum(
            [p.numel() for p in self.parameters() if p.requires_grad]
        )

        result_info = super().__str__()
        result_info = result_info + f"\nAll parameters: {all_parameters}"
        result_info = result_info + f"\nTrainable parameters: {trainable_parameters}"

        return result_info

In [ ]:
teacher_model = torch.load('model_checkpoint1.pt')
teacher_model.eval()

student_model = BERTLikeModel(
    vocab_size=28996,
    max_position_embeddings=128,
    hidden_dim=396,         # Размер скрытого слоя
    num_heads=6,            # Количество голов
    ff_dim=512,             # Размер feed-forward слоя
    num_layers=8,           # Количество слоев
    num_labels=9,           # Количество меток (для NER)
    dropout=0.1
)

In [ ]:
print(student_model)

In [ ]:
class KnowledgeDistillationLoss(nn.Module):
    def __init__(self, temperature=2.0, alpha=0.5):
        super().__init__()
        self.temperature = temperature
        self.alpha = alpha
        self.kl_loss = nn.KLDivLoss(reduction="batchmean")
        self.ce_loss = nn.CrossEntropyLoss()

    def forward(self, student_logits, teacher_logits, labels):
        student_probs = torch.log_softmax(student_logits / self.temperature, dim=-1)
        teacher_probs = torch.softmax(teacher_logits / self.temperature, dim=-1)        
        soft_loss = self.kl_loss(student_probs, teacher_probs)

        hard_loss = self.ce_loss(student_logits.view(-1, student_logits.size(-1)), labels.view(-1))    
        return self.alpha * soft_loss + (1 - self.alpha) * hard_loss, soft_loss, hard_loss

In [ ]:
train_loader = DataLoader(tokenized_datasets["train"], batch_size=128, shuffle=True, collate_fn=default_data_collator)
eval_loader = DataLoader(tokenized_datasets["validation"], batch_size=128, collate_fn=default_data_collator)

temperature=2.0
distillation_loss = KnowledgeDistillationLoss(temperature=temperature, alpha=0.5)

optimizer = AdamW(student_model.parameters(), lr=3e-4)
num_training_steps = len(train_loader) * 100  # 100 эпох
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=1000, num_training_steps=num_training_steps)

In [ ]:
wandb.init(
    project="nlp_hw4_size_reduction",
    name='distilation'
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher_model = nn.DataParallel(teacher_model.to(device))
student_model = nn.DataParallel(student_model.to(device))

for epoch in range(100):
    print('Epoch:', epoch)
    student_model.train()

    for step, batch in tqdm(enumerate(train_loader), desc='training', total=len(train_loader)):
        log_step = len(train_loader) * epoch + step
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            teacher_logits = teacher_model(input_ids=input_ids, attention_mask=attention_mask).logits

        student_logits = student_model(input_ids=input_ids, attention_mask=attention_mask)

        loss, soft_loss, hard_loss = distillation_loss(student_logits, teacher_logits, labels)

        loss = loss * temperature ** 2

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        wandb.log({
            "train_loss": loss.item(),
            "train_soft_loss": soft_loss.item(),
            "train_hard_loss": hard_loss.item(),
            "learning_rate": lr_scheduler.get_last_lr()[0]})

    student_model.eval()
    true_labels, predictions = [], []

    for step, batch in tqdm(enumerate(eval_loader), desc='validating', total=len(eval_loader)):
        log_step = len(train_loader) * epoch + step
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            logits = student_model(input_ids=input_ids, attention_mask=attention_mask)

        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        for label, pred in zip(labels, preds):
            true_labels.append([label_names[l] for l in label if l != -100])
            predictions.append([label_names[p] for l, p in zip(label, pred) if l != -100])

    f1 = f1_score(true_labels, predictions)
    wandb.log({"val_f1": f1})

wandb.finish()

In [ ]:
student_model.eval()
test_loader = DataLoader(tokenized_datasets["test"], batch_size=64, collate_fn=default_data_collator)
true_labels, predictions = [], []

for step, batch in tqdm(enumerate(test_loader), desc='testing', total=len(test_loader)):
    log_step = len(train_loader) * epoch + step
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)

    with torch.no_grad():
        logits = student_model(input_ids=input_ids, attention_mask=attention_mask)

    preds = torch.argmax(logits, dim=-1).cpu().numpy()
    for label, pred in zip(labels, preds):
        true_labels.append([label_names[l] for l in label if l != -100])
        predictions.append([label_names[p] for l, p in zip(label, pred) if l != -100])

f1 = f1_score(true_labels, predictions)
print(f"F1 on test: {f1:.4f}")

### Шеринг весов обучение с нуля

__Шеринг весов (2 балла).__ В модификации BERT [ALBERT](https://arxiv.org/pdf/1909.11942.pdf) помимо факторизации эмбеддингов предлагается шерить веса между слоями. То есть разные слои используют одни и те же веса. Такая техника эвивалентна применению одного и того же слоя несколько раз. Она позволяет в несколько раз уменьшить число параметров и не сильно потерять в качестве.

In [ ]:
class WeightSharingBERTLikeModel(nn.Module):
    def __init__(self, vocab_size, max_position_embeddings, hidden_dim, num_heads, ff_dim, num_layers, num_labels, dropout=0.1):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, hidden_dim)
        self.position_embeddings = nn.Embedding(max_position_embeddings, hidden_dim)
        self.layer_norm = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(dropout)

        shared_layer = TransformerEncoderLayer(hidden_dim, num_heads, ff_dim, dropout)

        self.encoder_layers = nn.ModuleList(
            [shared_layer for _ in range(num_layers)]
        )

        self.classifier = nn.Linear(hidden_dim, num_labels)

    def forward(self, input_ids, attention_mask=None):
        batch_size, seq_len = input_ids.size()
        position_ids = torch.arange(seq_len, device=input_ids.device).unsqueeze(0).expand(batch_size, seq_len)

        x = self.embeddings(input_ids) + self.position_embeddings(position_ids)
        x = self.layer_norm(x)
        x = self.dropout(x)

        for layer in self.encoder_layers:
            x = layer(x, attention_mask)

        logits = self.classifier(x)
        return logits
    
    def __str__(self):
        """
        Model prints with the number of parameters.
        """
        all_parameters = sum([p.numel() for p in self.parameters()])
        trainable_parameters = sum(
            [p.numel() for p in self.parameters() if p.requires_grad]
        )

        result_info = super().__str__()
        result_info = result_info + f"\nAll parameters: {all_parameters}"
        result_info = result_info + f"\nTrainable parameters: {trainable_parameters}"

        return result_info

In [ ]:
# teacher_model = torch.load('model_checkpoint1.pt')
# teacher_model.eval()

student_model = WeightSharingBERTLikeModel(
    vocab_size=28996,
    max_position_embeddings=128,
    hidden_dim=534,         # Размер скрытого слоя
    num_heads=6,            # Количество голов
    ff_dim=3072,             # Размер feed-forward слоя
    num_layers=8,           # Количество слоев
    num_labels=9,           # Количество меток (для NER)
    dropout=0.1
)

In [ ]:
print(student_model)

In [ ]:
train_loader = DataLoader(tokenized_datasets["train"], batch_size=32, shuffle=True, collate_fn=default_data_collator)
eval_loader = DataLoader(tokenized_datasets["validation"], batch_size=64, collate_fn=default_data_collator)

temperature=2.0
distillation_loss = KnowledgeDistillationLoss(temperature=temperature, alpha=0.5)

optimizer = AdamW(student_model.parameters(), lr=5e-5)
num_training_steps = len(train_loader) * 46  # 46 эпох
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=1000, num_training_steps=num_training_steps)

In [ ]:
wandb.init(
    project="nlp_hw4_size_reduction",
    name='distilation_with_sharing'
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher_model = nn.DataParallel(teacher_model.to(device))
student_model = nn.DataParallel(student_model.to(device))

for epoch in range(46):
    print('Epoch:', epoch)
    student_model.train()

    for step, batch in tqdm(enumerate(train_loader), desc='training', total=len(train_loader)):
        log_step = len(train_loader) * epoch + step
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            teacher_logits = teacher_model(input_ids=input_ids, attention_mask=attention_mask).logits

        student_logits = student_model(input_ids=input_ids, attention_mask=attention_mask)

        loss, soft_loss, hard_loss = distillation_loss(student_logits, teacher_logits, labels)

        loss = loss * temperature ** 2

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        wandb.log({
            "train_loss": loss.item(),
            "train_soft_loss": soft_loss.item(),
            "train_hard_loss": hard_loss.item(),
            "learning_rate": lr_scheduler.get_last_lr()[0]})

    student_model.eval()
    true_labels, predictions = [], []

    for step, batch in tqdm(enumerate(eval_loader), desc='validating', total=len(eval_loader)):
        log_step = len(train_loader) * epoch + step
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            logits = student_model(input_ids=input_ids, attention_mask=attention_mask)

        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        for label, pred in zip(labels, preds):
            true_labels.append([label_names[l] for l in label if l != -100])
            predictions.append([label_names[p] for l, p in zip(label, pred) if l != -100])

    f1 = f1_score(true_labels, predictions)
    wandb.log({"val_f1": f1})

wandb.finish()

In [ ]:
student_model.eval()
test_loader = DataLoader(tokenized_datasets["test"], batch_size=64, collate_fn=default_data_collator)
true_labels, predictions = [], []

for step, batch in tqdm(enumerate(test_loader), desc='testing', total=len(test_loader)):
    log_step = len(train_loader) * epoch + step
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)

    with torch.no_grad():
        logits = student_model(input_ids=input_ids, attention_mask=attention_mask)

    preds = torch.argmax(logits, dim=-1).cpu().numpy()
    for label, pred in zip(labels, preds):
        true_labels.append([label_names[l] for l in label if l != -100])
        predictions.append([label_names[p] for l, p in zip(label, pred) if l != -100])

f1 = f1_score(true_labels, predictions)
print(f"F1 on test: {f1:.4f}")

### Шеринг весов ft

In [ ]:
model = torch.load('model_checkpoint1.pt', map_location=torch.device('cpu'))

shared_layer = model.bert.encoder.layer[0]

for i in range(len(model.bert.encoder.layer)):
    model.bert.encoder.layer[i] = shared_layer

rank=420
original_embeddings = model.bert.embeddings.word_embeddings
factorized_embeddings = FactorizedEmbedding(original_embeddings, rank)

model.bert.embeddings.word_embeddings = factorized_embeddings

print('Число параметров:', sum(p.numel() for p in model.parameters()))

Число параметров: 19991961


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
wandb.init(
    project="nlp_hw4_size_reduction",
    name='sharing_encoder_weights_ft'
)

trainer.train()

In [ ]:
test_results = trainer.evaluate(tokenized_datasets["test"])
print(test_results)

wandb.finish()

### Факторизация промежуточных слоев обучение с нуля

__Факторизация промежуточных слоев (2 балла).__ Если можно факторизовать матрицу эмбеддингов, то и все остальное тоже можно. Для факторизации слоев существует много разных подходов и выбрать какой-то один сложно. Вы можете вдохновляться [этим списком](https://lechnowak.com/posts/neural-network-low-rank-factorization-techniques/), найти в интернете что-то другое или придумать метод самостоятельно. В любом случае в отчете обоснуйте, почему вы решили сделать так как сделали.

In [42]:
class FactorizedLinear(nn.Module):
    def __init__(self, in_features, out_features, rank):
        super().__init__()
        self.U = nn.Linear(in_features, rank, bias=False)
        self.V = nn.Linear(rank, out_features, bias=True)

    def forward(self, x):
        return self.V(self.U(x))


class FactorizedMultiHeadAttention(MultiHeadAttention):
    def __init__(self, hidden_dim, num_heads, rank, dropout=0.1):
        super().__init__(hidden_dim, num_heads, dropout)
        self.query = FactorizedLinear(hidden_dim, hidden_dim, rank)
        self.key = FactorizedLinear(hidden_dim, hidden_dim, rank)
        self.value = FactorizedLinear(hidden_dim, hidden_dim, rank)
        self.out = FactorizedLinear(hidden_dim, hidden_dim, rank)


class FactorizedFeedForward(FeedForward):
    def __init__(self, hidden_dim, ff_dim, rank, dropout=0.1):
        super().__init__(hidden_dim, ff_dim, dropout)
        self.linear1 = FactorizedLinear(hidden_dim, ff_dim, rank)
        self.linear2 = FactorizedLinear(ff_dim, hidden_dim, rank)


class FactorizedTransformerEncoderLayer(TransformerEncoderLayer):
    def __init__(self, hidden_dim, num_heads, ff_dim, rank, dropout=0.1):
        super().__init__(hidden_dim, num_heads, ff_dim, dropout)
        self.attention = FactorizedMultiHeadAttention(hidden_dim, num_heads, rank, dropout)
        self.feed_forward = FactorizedFeedForward(hidden_dim, ff_dim, rank, dropout)


class FactorizedBERTLikeModel(BERTLikeModel):
    def __init__(self, vocab_size, max_position_embeddings, hidden_dim, num_heads, ff_dim, num_layers, num_labels, rank, dropout=0.1):
        super().__init__(vocab_size, max_position_embeddings, hidden_dim, num_heads, ff_dim, num_layers, num_labels, dropout)

        self.encoder_layers = nn.ModuleList(
            [FactorizedTransformerEncoderLayer(hidden_dim, num_heads, ff_dim, rank, dropout) for _ in range(num_layers)]
        )

        self.classifier = FactorizedLinear(hidden_dim, num_labels, rank)


In [ ]:
factorized_model = FactorizedBERTLikeModel(
    vocab_size=28996,
    max_position_embeddings=128,
    hidden_dim=512,         # Размер скрытого слоя
    num_heads=4,            # Количество голов
    ff_dim=2048,             # Размер feed-forward слоя
    num_layers=6,           # Количество слоев
    num_labels=9,           # Количество меток (например, для NER)
    rank=90,                # Ранг факторизации
    dropout=0.1
)

In [ ]:
print(factorized_model)

In [ ]:
train_loader = DataLoader(tokenized_datasets["train"], batch_size=128, shuffle=True, collate_fn=default_data_collator)
eval_loader = DataLoader(tokenized_datasets["validation"], batch_size=128, collate_fn=default_data_collator)

temperature=2.0
distillation_loss = KnowledgeDistillationLoss(temperature=temperature, alpha=0.5)

optimizer = AdamW(factorized_model.parameters(), lr=3e-4)
num_training_steps = len(train_loader) * 100  # 100 эпох
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=1000, num_training_steps=num_training_steps)

In [ ]:
wandb.init(
    project="nlp_hw4_size_reduction",
    name='distilation_with_all_factorization'
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher_model = nn.DataParallel(teacher_model.to(device))
factorized_model = nn.DataParallel(factorized_model.to(device))

for epoch in range(100):
    print('Epoch:', epoch)
    factorized_model.train()

    for step, batch in tqdm(enumerate(train_loader), desc='training', total=len(train_loader)):
        log_step = len(train_loader) * epoch + step
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            teacher_logits = teacher_model(input_ids=input_ids, attention_mask=attention_mask).logits

        factorized_logits = factorized_model(input_ids=input_ids, attention_mask=attention_mask)

        loss, soft_loss, hard_loss = distillation_loss(factorized_logits, teacher_logits, labels)

        loss = loss * temperature ** 2

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        wandb.log({
            "train_loss": loss.item(),
            "train_soft_loss": soft_loss.item(),
            "train_hard_loss": hard_loss.item(),
            "learning_rate": lr_scheduler.get_last_lr()[0]})

    factorized_model.eval()
    true_labels, predictions = [], []

    for step, batch in tqdm(enumerate(eval_loader), desc='validating', total=len(eval_loader)):
        log_step = len(train_loader) * epoch + step
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            logits = factorized_model(input_ids=input_ids, attention_mask=attention_mask)

        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        for label, pred in zip(labels, preds):
            true_labels.append([label_names[l] for l in label if l != -100])
            predictions.append([label_names[p] for l, p in zip(label, pred) if l != -100])

    f1 = f1_score(true_labels, predictions)
    wandb.log({"val_f1": f1})

wandb.finish()

In [ ]:
factorized_model.eval()
test_loader = DataLoader(tokenized_datasets["test"], batch_size=64, collate_fn=default_data_collator)
true_labels, predictions = [], []

for step, batch in tqdm(enumerate(test_loader), desc='testing', total=len(test_loader)):
    log_step = len(train_loader) * epoch + step
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)

    with torch.no_grad():
        logits = factorized_model(input_ids=input_ids, attention_mask=attention_mask)

    preds = torch.argmax(logits, dim=-1).cpu().numpy()
    for label, pred in zip(labels, preds):
        true_labels.append([label_names[l] for l in label if l != -100])
        predictions.append([label_names[p] for l, p in zip(label, pred) if l != -100])

f1 = f1_score(true_labels, predictions)
print(f"F1 on test: {f1:.4f}")

### Факторизаия ft

In [72]:
class FactorizedLinear(nn.Module):
    def __init__(self, in_features, out_features, rank):
        super().__init__()
        self.U = nn.Linear(in_features, rank, bias=False)
        self.V = nn.Linear(rank, out_features, bias=True) 

    def forward(self, x):
        return self.V(self.U(x))

def svd_factorize_linear(layer, rank):
    with torch.no_grad():
        W = layer.weight.data
        U, S, Vh = torch.linalg.svd(W, full_matrices=False)
        
        U_reduced = U[:, :rank]
        S_reduced = S[:rank]
        Vh_reduced = Vh[:rank, :]
        
        U_weight = U_reduced @ torch.diag(S_reduced)
        V_weight = Vh_reduced

        assert U.shape[0] == W.shape[0]
        assert Vh.shape[1] == W.shape[1]

        
        factorized_layer = FactorizedLinear(W.size(1), W.size(0), rank)

        factorized_layer.U.weight.data.copy_(V_weight)
        factorized_layer.V.weight.data.copy_(U_weight)
        
        if layer.bias is not None:
            factorized_layer.V.bias.data.copy_(layer.bias.data)

        return factorized_layer

def replace_linear_with_factorized(module, rank):
    for name, child in module.named_children():
        if isinstance(child, nn.Linear):
            change = True
            for shape in child.weight.data.shape:
                if shape < rank:
                    change = False
            if change:
                factorized_layer = svd_factorize_linear(child, rank)
                setattr(module, name, factorized_layer)
        else:
            replace_linear_with_factorized(child, rank)
    return module


In [ ]:
model = torch.load('model_checkpoint1.pt', map_location=torch.device('cpu'))

rank = 99
factorized_model = replace_linear_with_factorized(model, rank)
original_embeddings = factorized_model.bert.embeddings.word_embeddings
factorized_embeddings = FactorizedEmbedding(original_embeddings, rank)

factorized_model.bert.embeddings.word_embeddings = factorized_embeddings

print('Число параметров:', sum(p.numel() for p in factorized_model.parameters()))

Число параметров: 19892565


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
wandb.init(
    project="nlp_hw4_size_reduction",
    name='factorized_all_ft'
)

trainer.train()

In [ ]:
test_results = trainer.evaluate(tokenized_datasets["test"])
print(test_results)

wandb.finish()

### Приближение промежуточнх слоев

__Приближение промежуточных слоев (2 балла).__ Мы обсуждали, что помимо приближения выходов модели ученика к выходам модели учителя, можно приближать выходы промежуточных слоев. В [этой работе](https://www.researchgate.net/publication/375758425_Knowledge_Distillation_Scheme_for_Named_Entity_Recognition_Model_Based_on_BERT) подробно написано, как это можно сделать.

In [13]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, mask=None):
        d_k = query.size(-1)
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            mask = mask[:, None, :, None] * mask[:, None, None, :]
            ones = torch.ones(mask.shape[-2:], device=mask.device)
            mask = mask + ones
            scores = scores.masked_fill(mask == 0, -1e9)
        attention = torch.softmax(scores, dim=-1)
        attention = self.dropout(attention)
        return torch.matmul(attention, value), attention


class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_dim, num_heads, dropout=0.1):
        super().__init__()
        assert hidden_dim % num_heads == 0, "hidden_dim must be divisible by num_heads"
        self.d_k = hidden_dim // num_heads
        self.num_heads = num_heads

        self.query = nn.Linear(hidden_dim, hidden_dim)
        self.key = nn.Linear(hidden_dim, hidden_dim)
        self.value = nn.Linear(hidden_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim, hidden_dim)

        self.attention = ScaledDotProductAttention(dropout)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(hidden_dim)

    def forward(self, x, mask=None):
        batch_size, seq_len, hidden_dim = x.size()

        query = self.query(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        key = self.key(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        value = self.value(x).view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)

        attn_output, attention_matrix = self.attention(query, key, value, mask)
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_len, hidden_dim)
        output = self.out(attn_output)

        return self.layer_norm(x + self.dropout(output)), attention_matrix


class FeedForward(nn.Module):
    def __init__(self, hidden_dim, ff_dim, dropout=0.1):
        super().__init__()
        self.linear1 = nn.Linear(hidden_dim, ff_dim)
        self.linear2 = nn.Linear(ff_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(hidden_dim)

    def forward(self, x):
        ff_output = self.linear2(self.dropout(torch.relu(self.linear1(x))))
        return self.layer_norm(x + self.dropout(ff_output))


class TransformerEncoderLayer(nn.Module):
    def __init__(self, hidden_dim, num_heads, ff_dim, dropout=0.1):
        super().__init__()
        self.attention = MultiHeadAttention(hidden_dim, num_heads, dropout)
        self.feed_forward = FeedForward(hidden_dim, ff_dim, dropout)

    def forward(self, x, mask=None):
        x, attention_matrix = self.attention(x, mask)
        return self.feed_forward(x), attention_matrix


class BERTLikeModel(nn.Module):
    def __init__(self, vocab_size, max_position_embeddings, hidden_dim, num_heads, ff_dim, num_layers, num_labels, dropout=0.1):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, hidden_dim)
        self.position_embeddings = nn.Embedding(max_position_embeddings, hidden_dim)
        self.layer_norm = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(dropout)

        self.encoder_layers = nn.ModuleList(
            [TransformerEncoderLayer(hidden_dim, num_heads, ff_dim, dropout) for _ in range(num_layers)]
        )

        self.classifier = nn.Linear(hidden_dim, num_labels)

    def forward(self, input_ids, attention_mask=None):
        batch_size, seq_len = input_ids.size()
        position_ids = torch.arange(seq_len, device=input_ids.device).unsqueeze(0).expand(batch_size, seq_len)

        x = self.embeddings(input_ids) + self.position_embeddings(position_ids)
        x = self.layer_norm(x)
        x = self.dropout(x)

        attentions, hidden_states = [], []

        for layer in self.encoder_layers:
            x, attention_matrix = layer(x, attention_mask)
            attentions.append(attention_matrix)
            hidden_states.append(x)

        logits = self.classifier(x)
        return logits, attentions, hidden_states
    
    def __str__(self):
        """
        Model prints with the number of parameters.
        """
        all_parameters = sum([p.numel() for p in self.parameters()])
        trainable_parameters = sum(
            [p.numel() for p in self.parameters() if p.requires_grad]
        )

        result_info = super().__str__()
        result_info = result_info + f"\nAll parameters: {all_parameters}"
        result_info = result_info + f"\nTrainable parameters: {trainable_parameters}"

        return result_info

In [17]:
teacher_model = AutoModelForTokenClassification.from_pretrained(
    'bert-base-cased',
    num_labels=len(label_names),
    output_hidden_states=True,
    output_attentions=True
)
teacher_model.eval()

student_hidden_dim =396
teacher_hidden_dim = teacher_model.bert.embeddings.word_embeddings.weight.shape[1]
student_model = BERTLikeModel(
    vocab_size=28996,
    max_position_embeddings=128,
    hidden_dim=student_hidden_dim,         # Размер скрытого слоя
    num_heads=12,            # Количество голов
    ff_dim=512,             # Размер feed-forward слоя
    num_layers=8,           # Количество слоев
    num_labels=9,           # Количество меток (для NER)
    dropout=0.1
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(student_model)

In [ ]:
print(teacher_model)


In [ ]:
class KnowledgeDistillationLoss(nn.Module):
    def __init__(self, device, temperature=2.0, alpha=0.5):
        super().__init__()
        self.device=device
        self.temperature = temperature
        self.alpha = alpha
        self.kl_loss = nn.KLDivLoss(reduction="batchmean")
        self.ce_loss = nn.CrossEntropyLoss()
        self.mse = nn.MSELoss()
        self.lin = nn.Linear(in_features=student_hidden_dim, out_features=teacher_hidden_dim, bias=False, device=device)

    def forward(self, student_logits, teacher_logits, student_attentions, teacher_attentions, student_hs, teacher_hs, labels):
        student_probs = torch.log_softmax(student_logits / self.temperature, dim=-1)
        teacher_probs = torch.softmax(teacher_logits / self.temperature, dim=-1)        
        soft_loss = self.kl_loss(student_probs, teacher_probs)
        # hard_loss = self.ce_loss(student_logits.view(-1, student_logits.size(-1)), labels.view(-1)) 
        hard_loss = torch.tensor([0], device=self.device)

        attn_loss = torch.mean(torch.tensor([self.mse(attn1, attn2) for attn1, attn2 in zip(student_attentions, teacher_attentions[::2])]))
        hs_loss = torch.mean(torch.tensor([self.mse(self.lin(hs1), hs2) for hs1, hs2 in zip(student_hs, teacher_hs[1::2])])) # remove embeddings hs

        trans_loss = attn_loss + hs_loss
        return self.alpha * (soft_loss + trans_loss) + (1 - self.alpha) * hard_loss, soft_loss, hard_loss, trans_loss

In [ ]:
train_loader = DataLoader(tokenized_datasets["train"], batch_size=128, shuffle=True, collate_fn=default_data_collator)
eval_loader = DataLoader(tokenized_datasets["validation"], batch_size=128, collate_fn=default_data_collator)

temperature=2.0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
distillation_loss = KnowledgeDistillationLoss(device, temperature=temperature, alpha=0.5)

optimizer = AdamW(student_model.parameters(), lr=3e-4)
num_training_steps = len(train_loader) * 100  # 100 эпох
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=1000, num_training_steps=num_training_steps)

In [ ]:
wandb.init(
    project="nlp_hw4_size_reduction",
    name='distilation_hs_attn_matching'
)

In [ ]:
teacher_model = nn.DataParallel(teacher_model.to(device))
student_model = nn.DataParallel(student_model.to(device))

for epoch in range(100):
    print('Epoch:', epoch)
    student_model.train()

    for step, batch in tqdm(enumerate(train_loader), desc='training', total=len(train_loader)):
        log_step = len(train_loader) * epoch + step
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            teacher_output = teacher_model(input_ids=input_ids, attention_mask=attention_mask)
            teacher_logits = teacher_output.logits
            teacher_attentions = teacher_output.attentions
            teacher_hs = teacher_output.hidden_states

        student_logits, student_attentions, student_hs = student_model(input_ids=input_ids, attention_mask=attention_mask)

        loss, soft_loss, hard_loss, trans_loss = distillation_loss(student_logits, teacher_logits, student_attentions, teacher_attentions, student_hs, teacher_hs, labels)

        loss = loss * temperature ** 2

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        wandb.log({
            "train_loss": loss.item(),
            "train_soft_loss": soft_loss.item(),
            "train_hard_loss": hard_loss.item(),
            "train_trans_loss": trans_loss.item(),
            "learning_rate": lr_scheduler.get_last_lr()[0]
        })

    student_model.eval()
    true_labels, predictions = [], []

    for step, batch in tqdm(enumerate(eval_loader), desc='validating', total=len(eval_loader)):
        log_step = len(train_loader) * epoch + step
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            logits, _, _ = student_model(input_ids=input_ids, attention_mask=attention_mask)

        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        for label, pred in zip(labels, preds):
            true_labels.append([label_names[l] for l in label if l != -100])
            predictions.append([label_names[p] for l, p in zip(label, pred) if l != -100])

    f1 = f1_score(true_labels, predictions)
    wandb.log({"val_f1": f1})

wandb.finish()

In [ ]:
student_model.eval()
test_loader = DataLoader(tokenized_datasets["test"], batch_size=64, collate_fn=default_data_collator)
true_labels, predictions = [], []

for step, batch in tqdm(enumerate(test_loader), desc='testing', total=len(test_loader)):
    log_step = len(train_loader) * epoch + step
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)

    with torch.no_grad():
        logits = student_model(input_ids=input_ids, attention_mask=attention_mask)

    preds = torch.argmax(logits, dim=-1).cpu().numpy()
    for label, pred in zip(labels, preds):
        true_labels.append([label_names[l] for l in label if l != -100])
        predictions.append([label_names[p] for l, p in zip(label, pred) if l != -100])

f1 = f1_score(true_labels, predictions)
print(f"F1 on test: {f1:.4f}")

2nd stage

In [ ]:
gdown.download(id='1CSa_rYgoUN-E2I3guM2r6P5F9JkZAr1v')
gdown.download(id='1YSAesbwk9Z_Z8Blw0cPBwpDYtMifAvNF')

In [ ]:
teacher_model = torch.load('ft_model_with_attn_hs.pt')
student_model = torch.load('student_model_pretrain.pt')

In [ ]:
class KnowledgeDistillationLoss(nn.Module):
    def __init__(self, device, temperature=2.0, alpha=0.5):
        super().__init__()
        self.device=device
        self.temperature = temperature
        self.alpha = alpha
        self.kl_loss = nn.KLDivLoss(reduction="batchmean")
        self.ce_loss = nn.CrossEntropyLoss()
        self.mse = nn.MSELoss()
        self.lin = nn.Linear(in_features=student_hidden_dim, out_features=teacher_hidden_dim, bias=False, device=device)

    def forward(self, student_logits, teacher_logits, student_attentions, teacher_attentions, student_hs, teacher_hs, labels):
        student_probs = torch.log_softmax(student_logits / self.temperature, dim=-1)
        teacher_probs = torch.softmax(teacher_logits / self.temperature, dim=-1)        
        soft_loss = self.kl_loss(student_probs, teacher_probs)
        hard_loss = self.ce_loss(student_logits.view(-1, student_logits.size(-1)), labels.view(-1)) 

        attn_loss = torch.sum(torch.tensor([self.mse(attn1, attn2) for attn1, attn2 in zip(student_attentions, teacher_attentions[::2])]))
        hs_loss = torch.sum(torch.tensor([self.mse(self.lin(hs1), hs2) for hs1, hs2 in zip(student_hs, teacher_hs[1::2])])) # remove embeddings hs

        trans_loss = attn_loss + hs_loss
        return self.alpha * (soft_loss + trans_loss) + (1 - self.alpha) * hard_loss, soft_loss, hard_loss, trans_loss

In [ ]:
train_loader = DataLoader(tokenized_datasets["train"], batch_size=128, shuffle=True, collate_fn=default_data_collator)
eval_loader = DataLoader(tokenized_datasets["validation"], batch_size=128, collate_fn=default_data_collator)

temperature=2.0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
distillation_loss = KnowledgeDistillationLoss(device, temperature=temperature, alpha=0.5)

optimizer = AdamW(list(student_model.parameters()) + list(distillation_loss.parameters()), lr=3e-4)
num_training_steps = len(train_loader) * 100  # 100 эпох
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=1000, num_training_steps=num_training_steps)

In [ ]:
wandb.init(
    project="nlp_hw4_size_reduction",
    name='distilation_hs_attn_matching_2_stage_sum'
)

In [ ]:
teacher_model = teacher_model.to(device)
student_model = student_model.to(device)

for epoch in range(100):
    print('Epoch:', epoch)
    student_model.train()

    for step, batch in tqdm(enumerate(train_loader), desc='training', total=len(train_loader)):
        log_step = len(train_loader) * epoch + step
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            teacher_output = teacher_model(input_ids=input_ids, attention_mask=attention_mask)
            teacher_logits = teacher_output.logits
            teacher_attentions = teacher_output.attentions
            teacher_hs = teacher_output.hidden_states

        student_logits, student_attentions, student_hs = student_model(input_ids=input_ids, attention_mask=attention_mask)

        loss, soft_loss, hard_loss, trans_loss = distillation_loss(student_logits, teacher_logits, student_attentions, teacher_attentions, student_hs, teacher_hs, labels)

        loss = loss * temperature ** 2

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        wandb.log({
            "train_loss": loss.item(),
            "train_soft_loss": soft_loss.item(),
            "train_hard_loss": hard_loss.item(),
            "train_trans_loss": trans_loss.item(),
            "learning_rate": lr_scheduler.get_last_lr()[0]
        })

    student_model.eval()
    true_labels, predictions = [], []

    for step, batch in tqdm(enumerate(eval_loader), desc='validating', total=len(eval_loader)):
        log_step = len(train_loader) * epoch + step
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            logits, _, _ = student_model(input_ids=input_ids, attention_mask=attention_mask)

        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        for label, pred in zip(labels, preds):
            true_labels.append([label_names[l] for l in label if l != -100])
            predictions.append([label_names[p] for l, p in zip(label, pred) if l != -100])

    f1 = f1_score(true_labels, predictions)
    wandb.log({"val_f1": f1})

wandb.finish()

In [ ]:
student_model.eval()

test_loader = DataLoader(tokenized_datasets["test"], batch_size=64, collate_fn=default_data_collator)

true_labels, predictions = [], []



for step, batch in tqdm(enumerate(test_loader), desc='testing', total=len(test_loader)):

    log_step = len(train_loader) * epoch + step

    input_ids = batch["input_ids"].to(device)

    attention_mask = batch["attention_mask"].to(device)

    labels = batch["labels"].to(device)



    with torch.no_grad():

        logits, _, _ = student_model(input_ids=input_ids, attention_mask=attention_mask)



    preds = torch.argmax(logits, dim=-1).cpu().numpy()

    for label, pred in zip(labels, preds):

        true_labels.append([label_names[l] for l in label if l != -100])

        predictions.append([label_names[p] for l, p in zip(label, pred) if l != -100])



f1 = f1_score(true_labels, predictions)

print(f"F1 on test: {f1:.4f}")

# Задания на выбор

Как вы понимаете, есть еще довольно много разных способов уменьшить обученную модель. В этой секции вам предлагается реализовать разные техники на выбор. За каждую из них можно получить разное количество балов в зависимости от сложности. Успешность реализации будет оцениваться как по коду, так и по качеству на тестовой выборке. Все баллы за это дз, которые вы наберете сверх 10, будут считаться бонусными.   
В задании 4 вы обучали модель с ограничением числа параметров в 20М. При реализации техник из этой секции придерживайтесь такого же ограничение. Это позволит честно сравнивать методы между собой и делать правильные выводы. Напишите в отчете обо всем, что вы попробовали.

* __Шеринг весов (2 балла).__ В модификации BERT [ALBERT](https://arxiv.org/pdf/1909.11942.pdf) помимо факторизации эмбеддингов предлагается шерить веса между слоями. То есть разные слои используют одни и те же веса. Такая техника эвивалентна применению одного и того же слоя несколько раз. Она позволяет в несколько раз уменьшить число параметров и не сильно потерять в качестве.
* __Факторизация промежуточных слоев (2 балла).__ Если можно факторизовать матрицу эмбеддингов, то и все остальное тоже можно. Для факторизации слоев существует много разных подходов и выбрать какой-то один сложно. Вы можете вдохновляться [этим списком](https://lechnowak.com/posts/neural-network-low-rank-factorization-techniques/), найти в интернете что-то другое или придумать метод самостоятельно. В любом случае в отчете обоснуйте, почему вы решили сделать так как сделали.
* __Приближение промежуточных слоев (2 балла).__ Мы обсуждали, что помимо приближения выходов модели ученика к выходам модели учителя, можно приближать выходы промежуточных слоев. В [этой работе](https://www.researchgate.net/publication/375758425_Knowledge_Distillation_Scheme_for_Named_Entity_Recognition_Model_Based_on_BERT) подробно написано, как это можно сделать.
* __Прунинг (4 балла).__ В методе [SparseGPT](https://arxiv.org/abs/2301.00774) предлагается подход, удаляющий веса модели один раз после обучения. При этом оказывается возможным удалить до половины всех весов без потери в качестве. Математика, стоящаяя за техникой, довольно сложная, однако общий подход простой – будем удалять веса в каждом слое по отдельности, при удалении части весов слоя, остальные веса будут перенастраиваться так, чтобы общий выход слоя не изменился.
* __Удаление голов (6 баллов).__ В данный момент мы используем все головы внимания, но ряд исследований показывает, что большинство из них можно выбросить без потери качества. В этой [статье](https://arxiv.org/pdf/1905.09418.pdf) предлагается подход, который добавляет гейты к механизму внимания, которые регулируют, какие головы участвуют в слое, а какие – нет. В процессе обучения гейты настраиваются так, чтобы большинство голов не использовалась. В конце обучения неиспользуемые головы можно удалить. За это задание дается много баллов, потому что в методе довольно сложная математика и подход плохо заводится. Если вы решитесь потратить на него свои силы, то в случае неудачи мы дадим промежуточные баллы, опираясь на отчет.   
__Совет:__ во время обучения внимательно следите за поведением гейтов. Если вы все сделали правильно, то они должны зануляться. Однако зануляются они не всегда сразу, им надо дать время и обучать модель подольше.